## Importacion de los datos

In [2]:
import pandas as pd

df_news=pd.read_csv('noticias_bitcoin_top3_diarias.csv')

In [ ]:
df_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3291 entries, 0 to 3290
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   fecha      3291 non-null   object
 1   titulo     3291 non-null   object
 2   url        3291 non-null   object
 3   contenido  3291 non-null   object
dtypes: object(4)
memory usage: 103.0+ KB


'Bitcoin (BTC) Price Posts Worst Q1 in a Decade, Raising Questions About Where the Cycle Stands'

In [ ]:
!pip install transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [15]:
df_news['contenido'][0]

"The price of bitcoin (BTC-USD), the world's largest cryptocurrency by market value, was trading down around 5% on Monday morning as global markets plunged amid tariff tensions.\nThat level is off the lows from earlier on Monday morning.\nCCC - CoinMark... [442 chars]"

## Entrenamiento del modelo para analisis de sentimientos

In [19]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F
import pandas as pd

# Cargar modelo y tokenizer
model_name = 'cardiffnlp/twitter-roberta-base-sentiment-latest'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Función para calcular el sentimiento de una frase
def get_sentiment_score(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    outputs = model(**inputs)
    probs = F.softmax(outputs.logits, dim=1).detach().numpy()[0]
    neg, neu, pos = probs
    score = -1 * neg + 0 * neu + 1 * pos
    return score

# Aplicar a la columna de títulos
df_news['score'] = df_news['titulo'].apply(get_sentiment_score)

# Mostrar resultados
print(df_news[['titulo', 'score']].head())


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


                                              titulo     score
0        Bitcoin price falls amid tariff market rout -0.536233
1  US crypto stocks plunge as bitcoin hits new 20... -0.637867
2  SpaceX tourists return to Earth after historic...  0.630636
3   BlackRock buys $66M in Bitcoin amid market crash -0.137903
4  Markets tumble as Trump launches ‘Unstoppable’... -0.489370


In [29]:
print(df_news['titulo'][100])
print(df_news['score'][100])

Crypto markets steady after Trump’s first policy move
0.51583064


## Exportacion del Modelo

In [30]:
pd.DataFrame(df_news).to_csv('noticias_bitcoin_sentimientos.csv', index=False)